***Вариант***

In [1]:
surname = "марчук" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  1
Задача № 2:  2


Датасеты

hadoop fs -mkdir /dataset/hw4/small

hdfs dfs -put /home/ubuntu/_practice/hw4/small/links.csv /dataset/hw4/small/links.csv
hdfs dfs -put /home/ubuntu/_practice/hw4/small/movies.csv /dataset/hw4/small/movies.csv
hdfs dfs -put /home/ubuntu/_practice/hw4/small/ratings.csv /dataset/hw4/small/ratings.csv
hdfs dfs -put /home/ubuntu/_practice/hw4/small/tags.csv /dataset/hw4/small/tags.csv

In [2]:
# зависимости из дз1
import os

# пути к Java и Spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/ubuntu/_practice/spark-3.5.4-bin-hadoop3"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["SPARK_HOME"], "bin")

import findspark
findspark.init()

import pyspark
print(pyspark.__version__)

from pyspark import SparkContext, SparkConf

3.5.4


**Задание 1. Анализ датасета**

Вариант 1. Animation, Romance, Documentary

⚠️ Замечание: Один фильм может принадлежать разным жанрам

1. Выведите данные, сопоставляющие жанры и количество фильмов
2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, mean, desc, asc
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Инициализация SparkSession
spark = SparkSession.builder.appName("Movies Marchuk").getOrCreate()

# Пути к файлам в HDFS
movies_path = "/dataset/hw4/small/movies.csv"
ratings_path = "/dataset/hw4/small/ratings.csv"

# Загрузка данных
movies_df = spark.read.csv(movies_path, header=True, inferSchema=True)
ratings_df = spark.read.csv(ratings_path, header=True, inferSchema=True)

# Разделение жанров на отдельные строки
movies_with_genres = movies_df.withColumn("genre", explode(split(col("genres"), "\\|")))

# Фильтрация по целевым жанрам
target_genres = ["Animation", "Romance", "Documentary"]
movies_target_genres = movies_with_genres.filter(col("genre").isin(target_genres))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/13 23:11:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [4]:
# 1. Сопоставление жанров и количества фильмов
genre_counts = movies_target_genres.groupBy("genre").agg(count("*").alias("movie_count"))
genre_counts.show()

# Присоединение рейтингов
movies_ratings = movies_target_genres.join(ratings_df, "movieId")

+-----------+-----------+
|      genre|movie_count|
+-----------+-----------+
|    Romance|       1596|
|Documentary|        440|
|  Animation|        611|
+-----------+-----------+



In [5]:
# 2. Топ-10 фильмов с наибольшим количеством рейтингов для каждого жанра

# Создаем окно для нумерации фильмов в каждом жанре
window_spec = Window.partitionBy("genre").orderBy(col("rating_count").desc())

# Добавляем колонку с порядковым номером фильма в жанре
ranked_movies = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(count("rating").alias("rating_count"))
    .withColumn("rank", row_number().over(window_spec))
)

# Фильтруем, оставляя только топ-10 фильмов в каждом жанре
top_10_rated_per_genre = ranked_movies.filter(col("rank") <= 10)

# Сортируем и отображаем результаты
top_10_rated_per_genre = top_10_rated_per_genre.orderBy("genre", "rank")
top_10_rated_per_genre.show(100, truncate=False)

+-----------+-------+-------------------------------------------------------+------------+----+
|genre      |movieId|title                                                  |rating_count|rank|
+-----------+-------+-------------------------------------------------------+------------+----+
|Animation  |1      |Toy Story (1995)                                       |215         |1   |
|Animation  |588    |Aladdin (1992)                                         |183         |2   |
|Animation  |364    |Lion King, The (1994)                                  |172         |3   |
|Animation  |4306   |Shrek (2001)                                           |170         |4   |
|Animation  |595    |Beauty and the Beast (1991)                            |146         |5   |
|Animation  |6377   |Finding Nemo (2003)                                    |141         |6   |
|Animation  |4886   |Monsters, Inc. (2001)                                  |132         |7   |
|Animation  |8961   |Incredibles, The (2

In [6]:
# 3. Топ-10 фильмов с наименьшим количеством рейтингов (>10) для каждого жанра
window_spec_rating_count = Window.partitionBy("genre").orderBy(col("rating_count").asc())
least_10_rated_per_genre = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(count("rating").alias("rating_count"))
    .filter(col("rating_count") > 10)  # Условие > 10
    .withColumn("rank", row_number().over(window_spec_rating_count))
    .filter(col("rank") <= 10)  # Топ-10 для каждого жанра
    .orderBy("genre", "rank")
)
least_10_rated_per_genre.show(100, truncate=False)

+-----------+-------+--------------------------------------------------------------------------+------------+----+
|genre      |movieId|title                                                                     |rating_count|rank|
+-----------+-------+--------------------------------------------------------------------------+------------+----+
|Animation  |55442  |Persepolis (2007)                                                         |11          |1   |
|Animation  |49274  |Happy Feet (2006)                                                         |11          |2   |
|Animation  |97225  |Hotel Transylvania (2012)                                                 |11          |3   |
|Animation  |8965   |Polar Express, The (2004)                                                 |11          |4   |
|Animation  |709    |Oliver & Company (1988)                                                   |11          |5   |
|Animation  |52435  |How the Grinch Stole Christmas! (1966)                     

In [7]:
# 4. Топ-10 фильмов с наибольшим средним рейтингом (>10 рейтингов) для каждого жанра
window_spec_avg_rating_desc = Window.partitionBy("genre").orderBy(col("avg_rating").desc())
top_10_avg_rated_per_genre = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(
        count("rating").alias("rating_count"),
        mean("rating").alias("avg_rating")
    )
    .filter(col("rating_count") > 10)  # Условие > 10 рейтингов
    .withColumn("rank", row_number().over(window_spec_avg_rating_desc))
    .filter(col("rank") <= 10)  # Топ-10 для каждого жанра
    .orderBy("genre", "rank")
)
top_10_avg_rated_per_genre.show(100, truncate=False)

+-----------+-------+--------------------------------------------------------------------------+------------+------------------+----+
|genre      |movieId|title                                                                     |rating_count|avg_rating        |rank|
+-----------+-------+--------------------------------------------------------------------------+------------+------------------+----+
|Animation  |3429   |Creature Comforts (1989)                                                  |12          |4.25              |1   |
|Animation  |55442  |Persepolis (2007)                                                         |11          |4.181818181818182 |2   |
|Animation  |5690   |Grave of the Fireflies (Hotaru no haka) (1988)                            |16          |4.15625           |3   |
|Animation  |5618   |Spirited Away (Sen to Chihiro no kamikakushi) (2001)                      |87          |4.155172413793103 |4   |
|Animation  |741    |Ghost in the Shell (Kôkaku kidôtai) (1995

In [8]:
# 5. Топ-10 фильмов с наименьшим средним рейтингом (>10 рейтингов) для каждого жанра
window_spec_avg_rating = Window.partitionBy("genre").orderBy(col("avg_rating").asc())
least_10_avg_rated_per_genre = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(
        count("rating").alias("rating_count"),
        mean("rating").alias("avg_rating")
    )
    .filter(col("rating_count") > 10)  # Условие > 10 рейтингов
    .withColumn("rank", row_number().over(window_spec_avg_rating))
    .filter(col("rank") <= 10)  # Топ-10 для каждого жанра
    .orderBy("genre", "rank")
)
least_10_avg_rated_per_genre.show(100, truncate=False)

+-----------+-------+----------------------------------------------------------------+------------+------------------+----+
|genre      |movieId|title                                                           |rating_count|avg_rating        |rank|
+-----------+-------+----------------------------------------------------------------+------------+------------------+----+
|Animation  |8907   |Shark Tale (2004)                                               |13          |2.3461538461538463|1   |
|Animation  |69644  |Ice Age: Dawn of the Dinosaurs (2009)                           |14          |2.607142857142857 |2   |
|Animation  |49274  |Happy Feet (2006)                                               |11          |2.6818181818181817|3   |
|Animation  |2123   |All Dogs Go to Heaven (1989)                                    |15          |2.7               |4   |
|Animation  |673    |Space Jam (1996)                                                |53          |2.707547169811321 |5   |
|Animati

In [9]:
# Остановка SparkSession
spark.stop()

**Задание 2. Коллаборативная фильтрация**

Вариант 2. По схожести объектов

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
3. Определите rmse для тестового подмножества

In [10]:
# 1. Подготовка данных. Разделение данных на обучающую и тестовую выборки, 
#    вычисление среднего значения рейтинга для обучающей выборки и RMSE для тестовой выборки.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, mean, desc, asc, lit, avg, sqrt
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.ml.evaluation import RegressionEvaluator


# Инициализация SparkSession
spark = SparkSession.builder.appName("Movies2 Marchuk").getOrCreate()

# Загрузка данных из HDFS
ratings = spark.read.csv("/dataset/hw4/small/ratings.csv", header=True, inferSchema=True)

# Разделение на train_init (80%) и test (20%)
train_init, test = ratings.randomSplit([0.8, 0.2], seed=42)

# Среднее значение рейтинга в train_init
mean_rating = train_init.select(avg("rating").alias("mean_rating")).collect()[0]["mean_rating"]

# Предсказание среднего значения для тестового набора
test_with_predictions = test.withColumn("prediction", lit(mean_rating))

# Вычисление RMSE для тестового подмножества
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_mean = evaluator.evaluate(test_with_predictions)

print(f"Среднее значение рейтинга в train_init: {mean_rating:.2f}")
print(f"RMSE при предсказании среднего рейтинга: {rmse_mean:.4f}")

Среднее значение рейтинга в train_init: 3.50
RMSE при предсказании среднего рейтинга: 1.0504


In [11]:
#2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. 
#   Для определения схожести используйте train_init, для расчета rmse - test


from pyspark.sql.functions import col, sqrt, sum as spark_sum
from pyspark.ml.evaluation import RegressionEvaluator

# 1. Создание матрицы рейтингов "userId x movieId" для train_init
ratings_matrix = (
    train_init.groupBy("userId", "movieId")
    .agg(mean("rating").alias("rating"))
)

# 2. Вычисление косинусного сходства между фильмами
ratings_self_join = ratings_matrix.alias("r1").join(
    ratings_matrix.alias("r2"),
    col("r1.userId") == col("r2.userId")  # Сравнение по одному и тому же пользователю
)

# Подсчет числителя (скалярное произведение) и знаменателя (длины векторов)
movie_similarity = (
    ratings_self_join.groupBy("r1.movieId", "r2.movieId")
    .agg(
        spark_sum(col("r1.rating") * col("r2.rating")).alias("dot_product"),
        sqrt(spark_sum(col("r1.rating")**2)).alias("norm_r1"),
        sqrt(spark_sum(col("r2.rating")**2)).alias("norm_r2"),
    )
    .withColumn("similarity", col("dot_product") / (col("norm_r1") * col("norm_r2")))
    .filter(col("r1.movieId") != col("r2.movieId"))  # Убираем сравнение фильма с самим собой
)
# 3. Генерация предсказаний
# Для каждого фильма из test находим его ближайших соседей в train_init
predictions = (
    test.alias("t").join(
        movie_similarity.select(
            col("r1.movieId").alias("movieId_test"),  # Переименуем столбцы для удобства
            col("r2.movieId").alias("movieId_train"),
            "similarity"
        ).alias("ms"),
        col("t.movieId") == col("ms.movieId_test"),  # Связываем фильмы из тестового множества с похожими
        "left"
    )
    .join(
        train_init.alias("tr"),
        col("ms.movieId_train") == col("tr.movieId"),  # Связываем с рейтингами соседей
        "left"
    )
    .groupBy("t.userId", "t.movieId")
    .agg(
        spark_sum(col("ms.similarity") * col("tr.rating")).alias("weighted_sum"),
        spark_sum(col("ms.similarity")).alias("similarity_sum"),
    )
    .withColumn("prediction", col("weighted_sum") / col("similarity_sum"))
    .select("userId", "movieId", "prediction")
)

# 4. Оценка качества модели (RMSE)
# Объединяем предсказания с реальными рейтингами
test_with_predictions = test.join(predictions, ["userId", "movieId"], "left")

# Заполняем пропущенные значения средним рейтингом (если фильм не имеет похожих)
test_with_predictions = test_with_predictions.fillna(mean_rating, subset=["prediction"])

# Вычисление RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_collaborative = evaluator.evaluate(test_with_predictions)

print(f"RMSE для коллаборативной фильтрации: {rmse_collaborative:.4f}")

25/01/13 23:11:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:11:30 WARN RowBasedKeyValueBatch: Calling spill() on

RMSE для коллаборативной фильтрации: 1.0545


In [12]:
# 3. Определите rmse для тестового подмножества

from pyspark.ml.evaluation import RegressionEvaluator

# Оценка качества модели (RMSE)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Вычисление RMSE
rmse_collaborative = evaluator.evaluate(test_with_predictions)

print(f"RMSE для коллаборативной фильтрации: {rmse_collaborative:.4f}")

25/01/13 23:12:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/13 23:12:28 WARN RowBasedKeyValueBatch: Calling spill() on

RMSE для коллаборативной фильтрации: 1.0545


In [13]:
# Остановка SparkSession
spark.stop()

**Задание 3. Факторизация матрицы**
1. Выберите модель ALS по минимальному значению rmse. Для этого используйте кросс-валидацию k-folds c k=4

Параметры:

Количество факторов: [5, 10, 15]

Регуляризация: [0.001, 0.01, 0.1, 1, 10]

⚠️ Замечание: Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то rmse будет NaN

2. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов

In [14]:
# 1. Выберите модель ALS по минимальному значению rmse. Для этого используйте кросс-валидацию k-folds c k=4



from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
import numpy as np

# Инициализация SparkSession
spark = SparkSession.builder.appName("Movies3  Marchuk").getOrCreate()

# Загрузка данных
ratings = spark.read.csv("/dataset/hw4/small/ratings.csv", header=True, inferSchema=True)

# Параметры для кросс-валидации
k_folds = 4
factors = [5, 10, 15]
reg_params = [0.001, 0.01, 0.1, 1, 10]
seed = 42

# Функция для вычисления RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Кросс-валидация
min_rmse = float("inf")
best_model_params = None

# Создаем k фолдов
folds = ratings.randomSplit([1.0 / k_folds] * k_folds, seed=seed)

for factor in factors:
    for reg_param in reg_params:
        fold_rmse = []
        
        for i in range(k_folds):
            # Определяем обучающую и валидационную выборки
            validation = folds[i]
            train = spark.createDataFrame(
                [row for j, fold in enumerate(folds) if j != i for row in fold.collect()]
            )
            
            # Инициализация модели ALS
            als = ALS(
                maxIter=10,
                rank=factor,
                regParam=reg_param,
                userCol="userId",
                itemCol="movieId",
                ratingCol="rating",
                coldStartStrategy="drop",  # Убираем NaN предсказания
                seed=seed,
            )
            
            # Обучение модели
            model = als.fit(train)
            
            # Предсказания на валидационном наборе
            predictions = model.transform(validation)
            
            # Вычисление RMSE
            rmse = evaluator.evaluate(predictions)
            fold_rmse.append(rmse)
        
        # Среднее RMSE для текущих параметров
        avg_rmse = np.mean(fold_rmse)
        
        print(f"Factor: {factor}, RegParam: {reg_param}, RMSE: {avg_rmse:.4f}")
        
        # Сохранение лучших параметров
        if avg_rmse < min_rmse:
            min_rmse = avg_rmse
            best_model_params = {"factor": factor, "regParam": reg_param}

print(f"\nЛучшие параметры: {best_model_params}")
print(f"Минимальное RMSE: {min_rmse:.4f}")

25/01/13 23:13:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/01/13 23:13:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
                                                                                

Factor: 5, RegParam: 0.001, RMSE: 1.1978
Factor: 5, RegParam: 0.01, RMSE: 1.0453
Factor: 5, RegParam: 0.1, RMSE: 0.8875
Factor: 5, RegParam: 1, RMSE: 1.3214
Factor: 5, RegParam: 10, RMSE: 3.6638
Factor: 10, RegParam: 0.001, RMSE: 1.3454
Factor: 10, RegParam: 0.01, RMSE: 1.1476
Factor: 10, RegParam: 0.1, RMSE: 0.8915
Factor: 10, RegParam: 1, RMSE: 1.3214
Factor: 10, RegParam: 10, RMSE: 3.6638
Factor: 15, RegParam: 0.001, RMSE: 1.4202
Factor: 15, RegParam: 0.01, RMSE: 1.2101
Factor: 15, RegParam: 0.1, RMSE: 0.8918
Factor: 15, RegParam: 1, RMSE: 1.3214
Factor: 15, RegParam: 10, RMSE: 3.6638

Лучшие параметры: {'factor': 5, 'regParam': 0.1}
Минимальное RMSE: 0.8875


In [15]:
# Остановка SparkSession
spark.stop()

2. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов

Факторизация матрицы показала наилучший результат с минимальным RMSE 0.8875, что значительно лучше, чем предсказание среднего рейтинга и коллаборативная фильтрация по схожести объектов.
Однако, если доступно мало данных или вычислительные ресурсы ограничены, использование предсказания среднего или коллаборативной фильтрации может быть оправдано.
Для больших и сложных наборов данных корее всего больше подойдёт ALS, так как он лучше масштабируется и точнее захватывает скрытые паттерны в данных.

.

.

.

.

**================================= БОЛЬШОЙ НАБОР ДАННЫХ =================================**

Датасеты

hadoop fs -mkdir /dataset/hw4/big

hdfs dfs -put /home/ubuntu/_practice/hw4/big/links.csv /dataset/hw4/big/links.csv
hdfs dfs -put /home/ubuntu/_practice/hw4/big/movies.csv /dataset/hw4/big/movies.csv
hdfs dfs -put /home/ubuntu/_practice/hw4/big/ratings.csv /dataset/hw4/big/ratings.csv
hdfs dfs -put /home/ubuntu/_practice/hw4/big/tags.csv /dataset/hw4/big/tags.csv

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, mean, desc, asc
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Инициализация SparkSession
spark = SparkSession.builder.appName("Movies Marchuk").getOrCreate()

# Пути к файлам в HDFS
movies_path = "/dataset/hw4/big/movies.csv"
ratings_path = "/dataset/hw4/big/ratings.csv"

# Загрузка данных
movies_df = spark.read.csv(movies_path, header=True, inferSchema=True)
ratings_df = spark.read.csv(ratings_path, header=True, inferSchema=True)

# Разделение жанров на отдельные строки
movies_with_genres = movies_df.withColumn("genre", explode(split(col("genres"), "\\|")))

# Фильтрация по целевым жанрам
target_genres = ["Animation", "Romance", "Documentary"]
movies_target_genres = movies_with_genres.filter(col("genre").isin(target_genres))

In [17]:
# 1. Сопоставление жанров и количества фильмов
genre_counts = movies_target_genres.groupBy("genre").agg(count("*").alias("movie_count"))
genre_counts.show()

# Присоединение рейтингов
movies_ratings = movies_target_genres.join(ratings_df, "movieId")

+-----------+-----------+
|      genre|movie_count|
+-----------+-----------+
|    Romance|      10172|
|Documentary|       9283|
|  Animation|       4579|
+-----------+-----------+



In [18]:
# 2. Топ-10 фильмов с наибольшим количеством рейтингов для каждого жанра

# Создаем окно для нумерации фильмов в каждом жанре
window_spec = Window.partitionBy("genre").orderBy(col("rating_count").desc())

# Добавляем колонку с порядковым номером фильма в жанре
ranked_movies = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(count("rating").alias("rating_count"))
    .withColumn("rank", row_number().over(window_spec))
)

# Фильтруем, оставляя только топ-10 фильмов в каждом жанре
top_10_rated_per_genre = ranked_movies.filter(col("rank") <= 10)

# Сортируем и отображаем результаты
top_10_rated_per_genre = top_10_rated_per_genre.orderBy("genre", "rank")
top_10_rated_per_genre.show(100, truncate=False)

[Stage 8:==========================================>                (5 + 2) / 7]

+-----------+-------+-------------------------------------------------------+------------+----+
|genre      |movieId|title                                                  |rating_count|rank|
+-----------+-------+-------------------------------------------------------+------------+----+
|Animation  |1      |Toy Story (1995)                                       |76813       |1   |
|Animation  |4306   |Shrek (2001)                                           |58529       |2   |
|Animation  |588    |Aladdin (1992)                                         |55791       |3   |
|Animation  |364    |Lion King, The (1994)                                  |53509       |4   |
|Animation  |4886   |Monsters, Inc. (2001)                                  |48441       |5   |
|Animation  |6377   |Finding Nemo (2003)                                    |48124       |6   |
|Animation  |595    |Beauty and the Beast (1991)                            |45404       |7   |
|Animation  |8961   |Incredibles, The (2

In [19]:
# 3. Топ-10 фильмов с наименьшим количеством рейтингов (>10) для каждого жанра
window_spec_rating_count = Window.partitionBy("genre").orderBy(col("rating_count").asc())
least_10_rated_per_genre = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(count("rating").alias("rating_count"))
    .filter(col("rating_count") > 10)  # Условие > 10
    .withColumn("rank", row_number().over(window_spec_rating_count))
    .filter(col("rank") <= 10)  # Топ-10 для каждого жанра
    .orderBy("genre", "rank")
)
least_10_rated_per_genre.show(100, truncate=False)

[Stage 15:=================================================>        (6 + 1) / 7]

+-----------+-------+-------------------------------------------------------------------------------------------------------+------------+----+
|genre      |movieId|title                                                                                                  |rating_count|rank|
+-----------+-------+-------------------------------------------------------------------------------------------------------+------------+----+
|Animation  |182155 |Donald's Penguin (1939)                                                                                |11          |1   |
|Animation  |182189 |The Pied Piper (1933)                                                                                  |11          |2   |
|Animation  |216819 |The Art of Skiing (1941)                                                                               |11          |3   |
|Animation  |251630 |Maggie Simpson in The Force Awakens from Its Nap (2021)                                                |11         

In [20]:
# 4. Топ-10 фильмов с наибольшим средним рейтингом (>10 рейтингов) для каждого жанра
window_spec_avg_rating_desc = Window.partitionBy("genre").orderBy(col("avg_rating").desc())
top_10_avg_rated_per_genre = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(
        count("rating").alias("rating_count"),
        mean("rating").alias("avg_rating")
    )
    .filter(col("rating_count") > 10)  # Условие > 10 рейтингов
    .withColumn("rank", row_number().over(window_spec_avg_rating_desc))
    .filter(col("rank") <= 10)  # Топ-10 для каждого жанра
    .orderBy("genre", "rank")
)
top_10_avg_rated_per_genre.show(100, truncate=False)

[Stage 22:=================================>                        (4 + 3) / 7]

+-----------+-------+-----------------------------------------------------------------------------+------------+------------------+----+
|genre      |movieId|title                                                                        |rating_count|avg_rating        |rank|
+-----------+-------+-----------------------------------------------------------------------------+------------+------------------+----+
|Animation  |163809 |Over the Garden Wall (2013)                                                  |1430        |4.256993006993007 |1   |
|Animation  |286897 |Spider-Man: Across the Spider-Verse (2023)                                   |528         |4.252840909090909 |2   |
|Animation  |256991 |Adventure Time: Elements (2017)                                              |12          |4.25              |3   |
|Animation  |5618   |Spirited Away (Sen to Chihiro no kamikakushi) (2001)                         |35375       |4.226035335689046 |4   |
|Animation  |249180 |Violet Evergarden: T

In [21]:
# 5. Топ-10 фильмов с наименьшим средним рейтингом (>10 рейтингов) для каждого жанра
window_spec_avg_rating = Window.partitionBy("genre").orderBy(col("avg_rating").asc())
least_10_avg_rated_per_genre = (
    movies_ratings.groupBy("genre", "movieId", "title")
    .agg(
        count("rating").alias("rating_count"),
        mean("rating").alias("avg_rating")
    )
    .filter(col("rating_count") > 10)  # Условие > 10 рейтингов
    .withColumn("rank", row_number().over(window_spec_avg_rating))
    .filter(col("rank") <= 10)  # Топ-10 для каждого жанра
    .orderBy("genre", "rank")
)
least_10_avg_rated_per_genre.show(100, truncate=False)

[Stage 29:=========================================>                (5 + 2) / 7]

+-----------+-------+--------------------------------------------------------+------------+------------------+----+
|genre      |movieId|title                                                   |rating_count|avg_rating        |rank|
+-----------+-------+--------------------------------------------------------+------------+------------------+----+
|Animation  |120222 |Foodfight! (2012)                                       |46          |0.9456521739130435|1   |
|Animation  |170903 |The Swan Princess Christmas (2012)                      |11          |1.1363636363636365|2   |
|Animation  |153564 |The Amazing Bulk (2012)                                 |15          |1.1666666666666667|3   |
|Animation  |145096 |Barbie & Her Sisters in the Great Puppy Adventure (2015)|78          |1.1923076923076923|4   |
|Animation  |151313 |Norm of the North (2016)                                |58          |1.5086206896551724|5   |
|Animation  |6371   |Pokémon Heroes (2003)                              

In [22]:
# Остановка SparkSession
spark.stop()

**Задание 2. Коллаборативная фильтрация**

Вариант 2. По схожести объектов

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
3. Определите rmse для тестового подмножества

In [29]:
# 1. Подготовка данных. Разделение данных на обучающую и тестовую выборки, 
#    вычисление среднего значения рейтинга для обучающей выборки и RMSE для тестовой выборки.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, mean, desc, asc, lit, avg, sqrt
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.ml.evaluation import RegressionEvaluator


# Инициализация SparkSession
spark = SparkSession.builder.appName("Movies2 Marchuk").getOrCreate()

# Загрузка данных из HDFS
ratings = spark.read.csv("/dataset/hw4/big/ratings.csv", header=True, inferSchema=True)

# Разделение на train_init (80%) и test (20%)
train_init, test = ratings.randomSplit([0.8, 0.2], seed=42)

# Среднее значение рейтинга в train_init
mean_rating = train_init.select(avg("rating").alias("mean_rating")).collect()[0]["mean_rating"]

# Предсказание среднего значения для тестового набора
test_with_predictions = test.withColumn("prediction", lit(mean_rating))

# Вычисление RMSE для тестового подмножества
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_mean = evaluator.evaluate(test_with_predictions)

print(f"Среднее значение рейтинга в train_init: {mean_rating:.2f}")
print(f"RMSE при предсказании среднего рейтинга: {rmse_mean:.4f}")

[Stage 5:==================================================>        (6 + 1) / 7]

Среднее значение рейтинга в train_init: 3.54
RMSE при предсказании среднего рейтинга: 1.0640


In [ ]:
#2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. 
#   Для определения схожести используйте train_init, для расчета rmse - test


from pyspark.sql.functions import col, sqrt, sum as spark_sum
from pyspark.ml.evaluation import RegressionEvaluator

# 1. Создание матрицы рейтингов "userId x movieId" для train_init
ratings_matrix = (
    train_init.groupBy("userId", "movieId")
    .agg(mean("rating").alias("rating"))
)

# 2. Вычисление косинусного сходства между фильмами
ratings_self_join = ratings_matrix.alias("r1").join(
    ratings_matrix.alias("r2"),
    col("r1.userId") == col("r2.userId")  # Сравнение по одному и тому же пользователю
)

# Подсчет числителя (скалярное произведение) и знаменателя (длины векторов)
movie_similarity = (
    ratings_self_join.groupBy("r1.movieId", "r2.movieId")
    .agg(
        spark_sum(col("r1.rating") * col("r2.rating")).alias("dot_product"),
        sqrt(spark_sum(col("r1.rating")**2)).alias("norm_r1"),
        sqrt(spark_sum(col("r2.rating")**2)).alias("norm_r2"),
    )
    .withColumn("similarity", col("dot_product") / (col("norm_r1") * col("norm_r2")))
    .filter(col("r1.movieId") != col("r2.movieId"))  # Убираем сравнение фильма с самим собой
)
# 3. Генерация предсказаний
# Для каждого фильма из test находим его ближайших соседей в train_init
predictions = (
    test.alias("t").join(
        movie_similarity.select(
            col("r1.movieId").alias("movieId_test"),  # Переименуем столбцы для удобства
            col("r2.movieId").alias("movieId_train"),
            "similarity"
        ).alias("ms"),
        col("t.movieId") == col("ms.movieId_test"),  # Связываем фильмы из тестового множества с похожими
        "left"
    )
    .join(
        train_init.alias("tr"),
        col("ms.movieId_train") == col("tr.movieId"),  # Связываем с рейтингами соседей
        "left"
    )
    .groupBy("t.userId", "t.movieId")
    .agg(
        spark_sum(col("ms.similarity") * col("tr.rating")).alias("weighted_sum"),
        spark_sum(col("ms.similarity")).alias("similarity_sum"),
    )
    .withColumn("prediction", col("weighted_sum") / col("similarity_sum"))
    .select("userId", "movieId", "prediction")
)

# 4. Оценка качества модели (RMSE)
# Объединяем предсказания с реальными рейтингами
test_with_predictions = test.join(predictions, ["userId", "movieId"], "left")

# Заполняем пропущенные значения средним рейтингом (если фильм не имеет похожих)
test_with_predictions = test_with_predictions.fillna(mean_rating, subset=["prediction"])

# Вычисление RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_collaborative = evaluator.evaluate(test_with_predictions)

print(f"RMSE для коллаборативной фильтрации: {rmse_collaborative:.4f}")

Для расчетов не хватает пространства на диске, больше выделить физически не могу :(

In [30]:
# Остановка SparkSession
spark.stop()

**Задание 3. Факторизация матрицы**
1. Выберите модель ALS по минимальному значению rmse. Для этого используйте кросс-валидацию k-folds c k=4

Параметры:

Количество факторов: [5, 10, 15]

Регуляризация: [0.001, 0.01, 0.1, 1, 10]

⚠️ Замечание: Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то rmse будет NaN

2. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов

In [31]:
# 1. Выберите модель ALS по минимальному значению rmse. Для этого используйте кросс-валидацию k-folds c k=4



from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
import numpy as np

# Инициализация SparkSession
spark = SparkSession.builder.appName("Movies3 Marchuk").getOrCreate()

# Загрузка данных
ratings = spark.read.csv("/dataset/hw4/big/ratings.csv", header=True, inferSchema=True)

# Параметры для кросс-валидации
k_folds = 4
factors = [5, 10, 15]
reg_params = [0.001, 0.01, 0.1, 1, 10]
seed = 42

# Функция для вычисления RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Кросс-валидация
min_rmse = float("inf")
best_model_params = None

# Создаем k фолдов
folds = ratings.randomSplit([1.0 / k_folds] * k_folds, seed=seed)

for factor in factors:
    for reg_param in reg_params:
        fold_rmse = []
        
        for i in range(k_folds):
            # Определяем обучающую и валидационную выборки
            validation = folds[i]
            train = spark.createDataFrame(
                [row for j, fold in enumerate(folds) if j != i for row in fold.collect()]
            )
            
            # Инициализация модели ALS
            als = ALS(
                maxIter=10,
                rank=factor,
                regParam=reg_param,
                userCol="userId",
                itemCol="movieId",
                ratingCol="rating",
                coldStartStrategy="drop",  # Убираем NaN предсказания
                seed=seed,
            )
            
            # Обучение модели
            model = als.fit(train)
            
            # Предсказания на валидационном наборе
            predictions = model.transform(validation)
            
            # Вычисление RMSE
            rmse = evaluator.evaluate(predictions)
            fold_rmse.append(rmse)
        
        # Среднее RMSE для текущих параметров
        avg_rmse = np.mean(fold_rmse)
        
        print(f"Factor: {factor}, RegParam: {reg_param}, RMSE: {avg_rmse:.4f}")
        
        # Сохранение лучших параметров
        if avg_rmse < min_rmse:
            min_rmse = avg_rmse
            best_model_params = {"factor": factor, "regParam": reg_param}

print(f"\nЛучшие параметры: {best_model_params}")
print(f"Минимальное RMSE: {min_rmse:.4f}")

Py4JJavaError: An error occurred while calling o9309.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at scala.collection.mutable.ResizableArray.ensureSize(ResizableArray.scala:106)
	at scala.collection.mutable.ResizableArray.ensureSize$(ResizableArray.scala:96)
	at scala.collection.mutable.ArrayBuffer.ensureSize(ArrayBuffer.scala:49)
	at scala.collection.mutable.ArrayBuffer.$plus$eq(ArrayBuffer.scala:85)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$2(SparkPlan.scala:449)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$3541/833111582.apply(Unknown Source)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.util.NextIterator.foreach(NextIterator.scala:21)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:449)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:448)
	at org.apache.

In [ ]:
То же самое, не хватает места(

In [32]:
# Остановка SparkSession
spark.stop()